# Fundamentos Teóricos de la Regresión Logística

La regresión logística es un método estadístico utilizado para modelar la probabilidad de que un evento binario ocurra. Es decir, se utiliza para problemas de clasificación donde la variable dependiente es categórica (por ejemplo, sobrevivir o no sobrevivir).

### Ecuación de la Regresión Logística
La regresión logística utiliza la función sigmoide para modelar probabilidades:

$$ \hat{y} = \frac{1}{1 + e^{-z}} $$

Donde:
- $\hat{y}$: Probabilidad estimada de que el evento ocurra.
- $z$: Combinación lineal de las variables independientes ($z = \beta_0 + \beta_1x_1 + \beta_2x_2 + \dots + \beta_nx_n$).

### Supuestos de la Regresión Logística
1. La variable dependiente es binaria.
2. Las observaciones son independientes entre sí.
3. No hay multicolinealidad entre las variables independientes.
4. Relación lineal entre las variables independientes y el logit de la variable dependiente.

### Fundamento Matemático
La regresión logística maximiza la verosimilitud de los datos observados. La función de verosimilitud es:

$$ L(\beta) = \prod_{i=1}^n \hat{y}_i^{y_i} (1 - \hat{y}_i)^{1 - y_i} $$

Donde $y_i$ son los valores observados y $\hat{y}_i$ son las probabilidades predichas. El objetivo es encontrar los coeficientes $\beta$ que maximizan esta función.

### Fundamento Estadístico
La regresión logística asume que el logit (logaritmo de las probabilidades) sigue una relación lineal con las variables independientes:

$$ \text{logit}(\hat{y}) = \ln\left(\frac{\hat{y}}{1 - \hat{y}}\right) = \beta_0 + \beta_1x_1 + \beta_2x_2 + \dots + \beta_nx_n $$

Esto permite interpretar los coeficientes como el cambio en el logit por unidad de cambio en la variable independiente.

# ¿Cómo funciona la Regresión Logística?

La regresión logística es un modelo de clasificación que predice la probabilidad de que una observación pertenezca a una clase específica. Utiliza la función sigmoide para transformar una combinación lineal de las variables independientes en una probabilidad.

---

## Pasos de la Regresión Logística

- **1. Transformar las probabilidades en logits:**  
  La regresión logística modela el logit (logaritmo de las probabilidades):

$$
  \text{logit}(\hat{y}) = \ln\left(\frac{\hat{y}}{1 - \hat{y}}\right)
$$

- **2. Ajustar los coeficientes:**  
  Los coeficientes $\beta$ se ajustan para maximizar la función de verosimilitud, que mide qué tan bien el modelo predice los datos observados.

- **3. Predecir probabilidades:**  
  Una vez ajustado el modelo, se calculan las probabilidades de pertenecer a la clase positiva:

$$
  \hat{y} = \frac{1}{1 + e^{-z}}
$$

- **4. Clasificar observaciones:**  
  Se utiliza un umbral (por defecto 0.5) para clasificar las observaciones en una clase u otra.

---

## Ejemplo práctico

Supón que estás prediciendo si un pasajero del Titanic sobrevivió o no basado en su edad, clase y género. El modelo ajustado podría ser:

$$
\text{logit}(\hat{y}) = -1.5 + 0.02 \cdot \text{edad} - 0.8 \cdot \text{clase} + 2.3 \cdot \text{género}
$$

Esto significa que:
- Cada año adicional de edad reduce ligeramente la probabilidad de sobrevivir.
- Estar en una clase más alta aumenta la probabilidad de sobrevivir.
- Ser mujer aumenta significativamente la probabilidad de sobrevivir.

---

Este proceso permite interpretar los coeficientes y predecir probabilidades para nuevas observaciones.

In [1]:
# Clonar el repositorio y cambiar de directorio
!git clone https://github.com/davidjamesknight/SQLite_databases_for_learning_data_science
%cd SQLite_databases_for_learning_data_science

fatal: destination path 'SQLite_databases_for_learning_data_science' already exists and is not an empty directory.
/Users/jorgeramirez/Documents/Python Projects/3 Modelos Predictivos/SQLite_databases_for_learning_data_science


In [2]:
# Conectar a la base de datos y realizar el query para obtener el dataset del Titanic
import sqlite3
import pandas as pd

conn = sqlite3.connect('titanic.db')

query = """
SELECT
    O.survived,
    O.pclass,
    O.age,
    O.sibsp,
    O.parch,
    O.fare,
    O.adult_male,
    O.alone,
    S.sex,
    E.embarked
FROM
    Observation AS O
JOIN
    Sex AS S ON O.sex_id = S.sex_id
JOIN
    Embarked AS E ON O.embarked_id = E.embarked_id
"""
df = pd.read_sql_query(query, conn)
df.head()

,survived,pclass,age,sibsp,parch,fare,adult_male,alone,sex,embarked
0,0,3,22.0,1,0,7.2500,1,0,male,S
1,1,1,38.0,1,0,71.2833,0,0,female,C
2,1,3,26.0,0,0,7.9250,0,1,female,S
3,1,1,35.0,1,0,53.1000,0,0,female,S
4,0,3,35.0,0,0,8.0500,1,1,male,S


In [3]:
# Dividir el dataset en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

X = df.drop(columns=['survived'])
y = df['survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42
)

## Aseguramiento de calidad de los datos

In [4]:
X_train.isnull().sum()

pclass          0
age           140
sibsp           0
parch           0
fare            0
adult_male      0
alone           0
sex             0
embarked        2
dtype: int64

### Imputar valores faltantes

In [5]:
import plotly.express as px

fig = px.histogram(
    X_train, 
    x='age', 
    nbins=30, 
    title=f'Distribución y Densidad de Edad', 
    marginal="box", 
opacity=0.7)

fig.update_layout(bargap=0.2)

fig.show()

Como la variable "Edad" está sesgada hacia la derecha, usar la media podría darnos resultados sesgados al rellenar edades mayores a las deseadas. Para solucionar esto, utilizaremos la mediana para imputar los valores faltantes.

In [6]:
porcentaje_faltantes = (X_train['embarked'].isnull().sum() / X_train.shape[0]) * 100
print(f'Porcentaje de faltantes en la columna "embarked" es {porcentaje_faltantes:.2f}%')

Porcentaje de faltantes en la columna "embarked" es 0.28%


In [7]:
# Mostrar conteo de pasajeros por puerto de embarque
print('Pasajeros embarcados agrupados por puerto (C = Cherburgo, Q = Queenstown, S = Southampton):')
print(X_train['embarked'].value_counts())

# Crear gráfico de barras con Plotly
fig = px.histogram(
    X_train, 
    x='embarked', 
    color='embarked',
    title='Distribución de pasajeros por puerto de embarque',
    labels={'embarked': 'Puerto de embarque'},
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.show()

Pasajeros embarcados agrupados por puerto (C = Cherburgo, Q = Queenstown, S = Southampton):
embarked
S    525
C    125
Q     60
Name: count, dtype: int64


In [8]:
puerto_mas_comun = X_train['embarked'].value_counts().idxmax()
print(f'El puerto de embarque más común es {puerto_mas_comun}.')

El puerto de embarque más común es S.


In [9]:
# Contar valores de supervivencia
conteo_supervivencia = y_train.value_counts().rename({0: 'No sobrevivió', 1: 'Sobrevivió'})

# Crear DataFrame para el gráfico
df_pie = pd.DataFrame({
    'Estado': conteo_supervivencia.index,
    'Cantidad': conteo_supervivencia.values
})

# Crear gráfico de pastel
fig = px.pie(df_pie, names='Estado', values='Cantidad',
             title='Distribución de supervivencia en el conjunto de entrenamiento',
             color_discrete_sequence=['lightcoral', 'darkturquoise'],
             hole=0.3)

fig.update_traces(textinfo='percent+label')
fig.show()

### Ajustes a los datos

In [10]:
train_data = X_train.copy()
train_data["age"].fillna(train_data["age"].median(skipna=True), inplace=True)
train_data["embarked"].fillna(train_data['embarked'].value_counts().idxmax(), inplace=True)

/var/folders/21/wbm0jm310h9b62ddpn50l29c0000gn/T/ipykernel_38426/118187576.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/var/folders/21/wbm0jm310h9b62ddpn50l29c0000gn/T/ipykernel_38426/118187576.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df

In [11]:
train_data.isnull().sum()

pclass        0
age           0
sibsp         0
parch         0
fare          0
adult_male    0
alone         0
sex           0
embarked      0
dtype: int64

### Cambios a otras variables

Según el diccionario de datos de Kaggle, tanto 'SibSp' como 'Parch' están relacionados con viajar en familia.
Para simplificar el análisis (y tener en cuenta la posible multicolinealidad), combinaremos el efecto de estas variables
en un solo predictor categórico: si el individuo viajaba solo o no.

In [12]:
import numpy as np

train_data['TravelAlone'] = np.where(
    (train_data["sibsp"]+train_data["parch"])>0,
    0,
    1
)

train_data.drop('sibsp', axis=1, inplace=True)
train_data.drop('parch', axis=1, inplace=True)

### Crear variables dummies

In [13]:
# Conteo de categorías en 'pclass' y 'embarked'
print("Conteo de categorías en la columna 'pclass':")
print(train_data['pclass'].value_counts())

print("\nConteo de categorías en la columna 'embarked':")
print(train_data['embarked'].value_counts())

print("\nConteo de categorías en la columna 'sex':")
print(train_data['sex'].value_counts())

Conteo de categorías en la columna 'pclass':
pclass
3    398
1    163
2    151
Name: count, dtype: int64

Conteo de categorías en la columna 'embarked':
embarked
S    527
C    125
Q     60
Name: count, dtype: int64

Conteo de categorías en la columna 'sex':
sex
male      467
female    245
Name: count, dtype: int64


In [14]:
# Vamos a obtener dummies ahora sin One-Hot Encoder sino con pd.get_dummies()

training = pd.get_dummies(
    train_data, 
    columns=["pclass", "embarked", "sex"]
)

training.drop('adult_male', axis=1, inplace=True)

final_train = training
final_train.head()

,age,fare,alone,TravelAlone,pclass_1,pclass_2,pclass_3,embarked_C,embarked_Q,embarked_S,sex_female,sex_male
331,45.5,28.5000,1,1,True,False,False,False,False,True,False,True
733,23.0,13.0000,1,1,False,True,False,False,False,True,False,True
382,32.0,7.9250,1,1,False,False,True,False,False,True,False,True
704,26.0,7.8542,0,0,False,False,True,False,False,True,False,True
813,6.0,31.2750,0,0,False,False,True,False,False,True,True,False


### Hacer lo mismo en el 'Test Data'

In [15]:
X_test.isnull().sum()

pclass         0
age           37
sibsp          0
parch          0
fare           0
adult_male     0
alone          0
sex            0
embarked       0
dtype: int64

In [16]:
# Se utiliza la información de Train Data para evitar 'data leakage'

test_data = X_test.copy()
test_data["age"].fillna(X_train["age"].median(skipna=True), inplace=True)

test_data['TravelAlone'] = np.where(
    (test_data["sibsp"]+test_data["parch"])>0,
    0, 
    1
)

test_data.drop('sibsp', axis=1, inplace=True)
test_data.drop('parch', axis=1, inplace=True)

testing = pd.get_dummies(test_data, columns=["pclass", "embarked", "sex"])
testing.drop('adult_male', axis=1, inplace=True)

final_test = testing
final_test.head()

/var/folders/21/wbm0jm310h9b62ddpn50l29c0000gn/T/ipykernel_38426/1763786055.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,age,fare,alone,TravelAlone,pclass_1,pclass_2,pclass_3,embarked_C,embarked_Q,embarked_S,sex_female,sex_male
709,28.0,15.2458,0,0,False,False,True,True,False,False,False,True
439,31.0,10.5000,1,1,False,True,False,False,False,True,False,True
840,20.0,7.9250,1,1,False,False,True,False,False,True,False,True
720,6.0,33.0000,0,0,False,True,False,False,False,True,True,False
39,14.0,11.2417,0,0,False,False,True,True,False,False,True,False


## Análisis Exploratorio

In [35]:
# Gráfico simple de la distribución de edad según supervivencia
import plotly.express as px

df = final_train.copy()
df['survived'] = y_train

fig = px.histogram(
    df, 
    x='age', 
    color='survived',
    nbins=30,
    barmode='overlay',
    labels={'survived': 'Supervivencia', 'age': 'Edad'},
    title='Distribución de edad según supervivencia'
)

fig.show()

#### Distribución de edad según supervivencia

La distribución de edad entre los sobrevivientes y los fallecidos es en realidad bastante similar.  
Una diferencia notable es que, entre los sobrevivientes, una mayor proporción eran niños.  
Todo indica que los pasajeros hicieron un esfuerzo por salvar a los niños, asignándoles un lugar en los botes salvavidas.

In [32]:

# Asegurar que los índices coincidan
final_train = final_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

# Combinar edad y supervivencia en un solo DataFrame
df = final_train.copy()
df['survived'] = y_train

# Agrupar por edad y calcular el promedio de supervivencia
avg_survival_by_age = df.groupby('age', as_index=False)['survived'].mean()

# Crear gráfico de barras
fig = px.bar(avg_survival_by_age, x='age', y='survived',
             title='Promedio de supervivencia por edad',
             labels={'age': 'Edad', 'survived': 'Tasa de supervivencia'},
             color_discrete_sequence=['LightSeaGreen'],
             width=1000, height=500)

fig.update_layout(xaxis=dict(range=[0, 85]))
fig.show()

#### Inclusión de variable categórica: "Menor"

Considerando la tasa de supervivencia de los pasajeros menores de 16 años, incluiré una nueva variable categórica en el conjunto de datos: **"menor"**.  
Esta variable permitirá identificar si un individuo era menor de edad al momento del embarque, lo cual podría estar relacionado con decisiones de rescate durante el naufragio.

In [33]:
final_train['IsMinor'] = np.where(
    final_train['age']<=16,
    1, 
    0
)

final_test['IsMinor'] = np.where(
    final_test['age']<=16, 
    1, 
    0
)

#### Exploración de la tarifa

In [36]:
# Visualización simple de la distribución de tarifas según supervivencia
import plotly.express as px

df = final_train.copy()
df['survived'] = y_train

fig = px.histogram(
    df,
    x='fare',
    color='survived',
    nbins=30,
    barmode='overlay',
    labels={'survived': 'Supervivencia', 'fare': 'Tarifa'},
    title='Distribución de tarifas según supervivencia',
    range_x=[0, 200]  # Limitar el rango para mejor visualización
)

fig.show()

#### Interpretación: relación entre tarifa y supervivencia

Dado que las distribuciones de tarifas son claramente diferentes entre los pasajeros que sobrevivieron y los que fallecieron, es probable que esta variable sea un predictor significativo en nuestro modelo final.  
Los pasajeros que pagaron tarifas más bajas parecen haber tenido menos probabilidades de sobrevivir.  
Esto probablemente está fuertemente correlacionado con la clase del pasajero, la cual exploraremos a continuación.

In [21]:
import plotly.express as px

def graficar_supervivencia_por_categoria(X_train, y_train, col_categorica):
    """
    Genera un gráfico de barras que muestra la tasa de supervivencia promedio
    según una variable categórica específica.

    Parámetros:
    - X_train: DataFrame con las variables explicativas
    - y_train: Serie o DataFrame con la variable 'survived'
    - col_categorica: string con el nombre de la columna categórica de interés
    """

    # Asegurar que los índices coincidan
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)

    # Combinar datos
    df = X_train.copy()
    df['survived'] = y_train

    # Agrupar por la variable categórica y calcular promedio de supervivencia
    promedio = df.groupby(col_categorica, as_index=False)['survived'].mean()

    # Títulos dinámicos
    titulo = f'Tasa de supervivencia por categoría: {col_categorica}'
    etiqueta_x = col_categorica.capitalize()
    etiqueta_y = 'Tasa de supervivencia'

    # Crear gráfico
    fig = px.bar(promedio, x=col_categorica, y='survived',
                 title=titulo,
                 labels={col_categorica: etiqueta_x, 'survived': etiqueta_y},
                 color_discrete_sequence=['darkturquoise'],
                 width=800, height=400)

    fig.show()

#### Exploración de la clase del pasajero

In [22]:
graficar_supervivencia_por_categoria(X_train, y_train, 'pclass')

##### Interpretación
Obviamente los de primera clase tendieron a sorbevivir más.

#### Exploración de puerto de embarque

In [23]:
graficar_supervivencia_por_categoria(X_train, y_train, 'embarked')

#### Supervivencia según puerto de embarque

Los pasajeros que embarcaron en Cherburgo, Francia, parecen haber tenido la mayor tasa de supervivencia.  
Los que embarcaron en Southampton tuvieron una probabilidad de supervivencia ligeramente menor que aquellos que embarcaron en Queenstown.  
Esto probablemente está relacionado con la clase del pasajero, o incluso con el orden de asignación de habitaciones (por ejemplo, es posible que los primeros pasajeros tuvieran habitaciones más cercanas a la cubierta).

#### Exploración de viajar solo o con familia

In [24]:
graficar_supervivencia_por_categoria(final_train, y_train, 'TravelAlone')

##### Supervivencia según compañía familiar

Los individuos que viajaban sin familia a bordo tuvieron más probabilidades de fallecer en el desastre que aquellos que viajaban acompañados.  
Dado el contexto histórico, es probable que muchos de los pasajeros que viajaban solos fueran hombres.

#### Exploración de la variable género

In [25]:
graficar_supervivencia_por_categoria(X_train, y_train, 'sex')

##### Interpretación de supervivencia por género
Claramente las mujeres sobrevivieron más

## Regresión Logística

### Selección de Características

## Eliminación recursiva de características (RFE)

Dado un estimador externo que asigna pesos a las variables, la **eliminación recursiva de características (RFE)** permite seleccionar las variables más relevantes considerando conjuntos cada vez más pequeños.

Primero, el estimador se entrena con el conjunto completo de variables, y se calcula la importancia de cada una mediante el atributo `coef_` o `feature_importances_`.  
Luego, se eliminan las variables menos importantes del conjunto actual.  
Este procedimiento se repite de forma recursiva sobre el conjunto reducido hasta alcanzar el número deseado de variables seleccionadas.

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

# Seleccionar columnas relevantes
columnas = [
    "age", 
    "fare", 
    "TravelAlone", 
    "pclass_1", 
    "pclass_2",
    # "pclass_3", 
    "embarked_C", 
    "embarked_S",
    # "embarked_Q", 
    "sex_male", 
    # "sex_female",
    "IsMinor"
]

X = final_train[columnas]
y = y_train  # 'survived' está en y_train

# Construir el modelo de regresión logística
modelo = LogisticRegression(max_iter=500)

# Aplicar RFE para seleccionar las 8 variables más importantes
rfe = RFE(modelo, n_features_to_select=8)
rfe = rfe.fit(X, y)

# Mostrar las variables seleccionadas
variables_seleccionadas = list(X.columns[rfe.support_])
print(f'Variables seleccionadas por RFE: {variables_seleccionadas}')

Variables seleccionadas por RFE: ['age', 'TravelAlone', 'pclass_1', 'pclass_2', 'embarked_C', 'embarked_S', 'sex_male', 'IsMinor']


##### Selección de características con RFE y validación cruzada

La técnica **RFECV** aplica la eliminación recursiva de características (RFE) dentro de un ciclo de validación cruzada para encontrar el número óptimo de variables a seleccionar.  
A continuación, se aplica RFE sobre un modelo de regresión logística, ajustando automáticamente la cantidad de variables seleccionadas mediante validación cruzada.

In [27]:
from sklearn.feature_selection import RFECV

# Aplicar RFECV con regresión logística
rfecv = RFECV(estimator=LogisticRegression(max_iter=500), step=1, cv=10, scoring='accuracy')
rfecv.fit(X, y)

# Mostrar resultados
print(f'Número óptimo de variables seleccionadas: {rfecv.n_features_}')
print(f'Variables seleccionadas: {list(X.columns[rfecv.support_])}')

# Obtener puntajes de validación cruzada desde cv_results_
scores = rfecv.cv_results_['mean_test_score']
num_features = list(range(1, len(scores) + 1))

# Crear gráfico interactivo
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=num_features,
    y=scores,
    mode='lines+markers',
    line=dict(color='darkturquoise', width=3),
    marker=dict(size=6),
    name='Puntaje de validación cruzada'
))

fig.update_layout(
    title='Optimización de selección de variables con RFECV',
    xaxis_title='Número de variables seleccionadas',
    yaxis_title='Puntaje de validación cruzada (accuracy)',
    width=900,
    height=500,
    template='plotly_white'
)

fig.show()

Número óptimo de variables seleccionadas: 8
Variables seleccionadas: ['age', 'TravelAlone', 'pclass_1', 'pclass_2', 'embarked_C', 'embarked_S', 'sex_male', 'IsMinor']


In [28]:
import plotly.figure_factory as ff

# Seleccionar variables
selected_features = ['age', 'TravelAlone', 'pclass_1', 'pclass_2', 'embarked_C', 
                     'embarked_S', 'sex_male', 'IsMinor']
X = final_train[selected_features]

# Calcular matriz de correlación
correlaciones = X.corr().round(2)

# Crear mapa de calor interactivo
fig = ff.create_annotated_heatmap(
    z=correlaciones.values,
    x=correlaciones.columns.tolist(),
    y=correlaciones.index.tolist(),
    colorscale='RdYlGn',
    showscale=True,
    reversescale=True,
    annotation_text=correlaciones.values,
    font_colors=['black'],
    hoverinfo='z'
)

fig.update_layout(
    title='Mapa de calor de correlaciones entre variables',
    width=800,
    height=600
)

fig.show()

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, log_loss, roc_curve, auc

# Entrenar modelo de regresión logística
X_train = final_train[selected_features]
X_test = final_test[selected_features]

modelo = LogisticRegression(max_iter=500)
modelo.fit(X_train, y_train)

# Predicciones
y_pred = modelo.predict(X_test)
y_pred_proba = modelo.predict_proba(X_test)[:, 1]

# Métricas
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

print("Resultados del modelo con división Train/Test:")
print(f"Precisión (accuracy): {accuracy_score(y_test, y_pred):.3f}")
print(f"Pérdida logística (log_loss): {log_loss(y_test, y_pred_proba):.3f}")
print(f"Área bajo la curva (AUC): {roc_auc:.3f}")

# Umbral con sensibilidad > 0.95
idx = np.min(np.where(tpr > 0.95))

# Gráfico ROC con Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines',
                         name=f'Curva ROC (AUC = {roc_auc:.3f})',
                         line=dict(color='coral', width=3)))

fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines',
                         name='Línea base',
                         line=dict(color='gray', dash='dash')))

# Líneas guía en el punto de sensibilidad > 0.95
fig.add_trace(go.Scatter(x=[fpr[idx], fpr[idx]], y=[0, tpr[idx]],
                         mode='lines', line=dict(color='blue', dash='dot'),
                         showlegend=False))
fig.add_trace(go.Scatter(x=[0, fpr[idx]], y=[tpr[idx], tpr[idx]],
                         mode='lines', line=dict(color='blue', dash='dot'),
                         showlegend=False))

fig.update_layout(title='Curva ROC - Regresión logística',
                  xaxis_title='Tasa de falsos positivos (1 - especificidad)',
                  yaxis_title='Tasa de verdaderos positivos (recall)',
                  width=900, height=500)

fig.show()

# Interpretación del umbral
print(f"Usar un umbral de {thresholds[idx]:.3f} garantiza una sensibilidad de {tpr[idx]:.3f}, "
      f"una especificidad de {1 - fpr[idx]:.3f}, "
      f"es decir, una tasa de falsos positivos de {fpr[idx] * 100:.2f}%.")

Resultados del modelo con división Train/Test:
Precisión (accuracy): 0.799
Pérdida logística (log_loss): 0.432
Área bajo la curva (AUC): 0.872


Usar un umbral de 0.098 garantiza una sensibilidad de 0.959, una especificidad de 0.371, es decir, una tasa de falsos positivos de 62.86%.


#### Validación Cruzada

In [30]:
from sklearn.model_selection import cross_val_score

# Crear el modelo de regresión logística
logreg = LogisticRegression(max_iter=500)

# Aplicar validación cruzada con 10 pliegues
# Se evalúan tres métricas: precisión, log_loss y AUC
puntajes_accuracy = cross_val_score(logreg, X_train, y_train, cv=10, scoring='accuracy')
puntajes_log_loss = cross_val_score(logreg, X_train, y_train, cv=10, scoring='neg_log_loss')
puntajes_auc = cross_val_score(logreg, X_train, y_train, cv=10, scoring='roc_auc')

# Mostrar resultados promedio
print("Resultados de validación cruzada (10 pliegues):")
print(f"{logreg.__class__.__name__} - Precisión promedio: {puntajes_accuracy.mean():.3f}")
print(f"{logreg.__class__.__name__} - Log loss promedio: {-puntajes_log_loss.mean():.3f}")
print(f"{logreg.__class__.__name__} - AUC promedio: {puntajes_auc.mean():.3f}")

Resultados de validación cruzada (10 pliegues):
LogisticRegression - Precisión promedio: 0.791
LogisticRegression - Log loss promedio: 0.467
LogisticRegression - AUC promedio: 0.835


## Evaluación del modelo de regresión logística

- Wl modelo de regresión logística tiene buen rendimiento general:
  - Clasifica correctamente casi 8 de cada 10 casos.
  - Tiene buena capacidad para distinguir entre sobrevivientes y no sobrevivientes.
  - Las probabilidades que genera están razonablemente bien calibradas.

- Si el objetivo es mejorar aún más:
  - Puedes probar ingeniería de variables adicional.
  - Ajustar hiperparámetros del modelo.
  - Comparar con otros algoritmos como Random Forest, Gradient Boosting, entre otros.